In [1]:
import httpx
import json

async def call_agent_api(prompt: str):
    url = "http://localhost:8000/chat"
    params = {"message": prompt}
    
    print(f"Enviando: {prompt}\n" + "-"*30)
    
    # Timeout=None é essencial para streams longos
    async with httpx.AsyncClient(timeout=None) as client:
        try:
            async with client.stream("GET", url, params=params) as response:
                if response.status_code != 200:
                    print(f"Erro na API: {response.status_code}")
                    return

                # Itera sobre cada linha do stream (SSE)
                async for line in response.aiter_lines():
                    if line.startswith("data: "):
                        # Remove o prefixo 'data: ' e carrega o JSON
                        try:
                            data = json.loads(line[6:])
                            
                            # Verifica se é um token ou o fim do stream
                            if "token" in data:
                                print(data["token"], end="", flush=True)
                            elif data.get("type") == "end":
                                print("\n" + "-"*30 + "\n[Stream Finalizado]")
                        except json.JSONDecodeError:
                            continue
        except Exception as e:
            print(f"\nErro de conexão: {e}")

In [2]:
# Agora você verá negrito, listas e códigos formatados enquanto a IA escreve
await call_agent_api("gere um texto de 20 caracteres em negrito, seguido por uma lista de 3 itens e um trecho de código em Python que imprime 'Olá, Mundo!'")

Enviando: gere um texto de 20 caracteres em negrito, seguido por uma lista de 3 itens e um trecho de código em Python que imprime 'Olá, Mundo!'
------------------------------

Erro de conexão: All connection attempts failed


In [1]:
import requests

url = "http://localhost:8000/agent/run"

payload = {
    "input": "Explique como funciona um sistema multi-agente com supervisor, executor e validator"
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers, timeout=60)

print("Status code:", response.status_code)
print("Response JSON:")
print(response.json())


Status code: 422
Response JSON:
{'detail': [{'type': 'missing', 'loc': ['query', 'input'], 'msg': 'Field required', 'input': None}]}
